*This notebook gets the raw traceroute measurement data from `curr_traceroute` table and filters the NATed and residential probe IDs, then inserts entire dataframe (probeid, timestamp, if_nat, if_residential, reverse_dns column) in `sk_one_off_extensions` table. Corresponding access_types derived from reverse_dns should be a separate notebook*
- - - 

In [1]:
import sqlite3
import pandas as pd
import requests
import ipaddress
import time

In [11]:
DB_LOCATION = 'lastmile.db'
CURR_TRACEROUTE = 'curr_traceroute'
SK_ONE_OFF_EXTENSIONS = 'sk_one_off_extensions'
METADATA_SNAPSHOT = 'metadata_snapshot'

In [39]:
con = sqlite3.connect(DB_LOCATION)

In [12]:
timestamp = int(time.time())
query1 = ''' SELECT  unit_id 
                   , dtime
                   , GROUP_CONCAT(hop, ';') as hop_no
                   , GROUP_CONCAT(hop_address, ';') as hop_address

             FROM      %s
             WHERE hop IN (1,2)
             GROUP BY  unit_id, dtime
         '''%(CURR_TRACEROUTE)

query  = ''' SELECT  unit_id as probeid
                   , %d as timestamp 
                   , public_ip
                   , hop_no
                   , hop_address
                   , '' as hop1
                   , '' as hop2
                   , 0  as if_residential
                   , 0  as if_nat
             FROM (%s) as a, %s as b 
             WHERE a.unit_id = b.id
             GROUP BY probeid
        '''%(timestamp, query1, METADATA_SNAPSHOT)

df = pd.read_sql(query, con)

In [13]:
df.tail()

,probeid,timestamp,public_ip,hop_no,hop_address,hop1,hop2,if_residential,if_nat
2204,584164,1430076405,83.216.94.110,1;2,192.168.15.1;,,,0,0
2205,584166,1430076405,83.216.95.26,1;2,192.168.15.1;,,,0,0
2206,589646,1430076405,81.108.189.199,1;2,192.168.0.1;81.108.189.1,,,0,0
2207,601700,1430076405,82.32.143.10,1;2,10.0.1.1;82.32.142.1,,,0,0
2208,608580,1430076405,31.185.194.101,1;2,192.168.1.254;195.166.128.188,,,0,0


In [14]:
df.count()

probeid           2209
timestamp         2209
public_ip         2209
hop_no            2209
hop_address       2209
hop1              2209
hop2              2209
if_residential    2209
if_nat            2209
dtype: int64

In [15]:
def get_hop1_address(froms):
    try: h1,_ = froms.split(';')
    except Exception as e: h1 = None
    return h1

def get_hop2_address(froms):
    try: _,h2 = froms.split(';')
    except Exception as e: h2 = None
    return h2

df['hop1'] = df['hop_address'].apply(get_hop1_address)
df['hop2'] = df['hop_address'].apply(get_hop2_address)

In [16]:
del df['hop_no']
del df['hop_address']

In [17]:
df.head()

,probeid,timestamp,public_ip,hop1,hop2,if_residential,if_nat
0,14190,1430076405,178.78.87.102,192.168.1.1,10.55.212.2,0,0
1,14266,1430076405,95.145.135.27,192.168.1.1,,0,0
2,14851,1430076405,86.153.18.233,192.168.1.254,172.16.13.6,0,0
3,14880,1430076405,151.230.25.209,192.168.0.1,,0,0
4,14905,1430076405,31.53.231.115,192.168.1.254,217.32.142.194,0,0


In [19]:
df.count()

probeid           2209
timestamp         2209
public_ip         2209
hop1              2205
hop2              2205
if_residential    2209
if_nat            2209
dtype: int64

In [20]:
def get_rdns_from_ip(ip):
    import socket
    try: rdns,_ = socket.getnameinfo( (str(ip), 0) ,0 )
    except Exception as e: print(e, type(e), ip)    
    else: print('.', end=''); return rdns 
df['reverse_dns'] = df['public_ip'].apply(get_rdns_from_ip)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [21]:
def check_if_nated(hop1):
    if hop1 is None or hop1 == 'None': return None
    try: ip = ipaddress.ip_address(hop1.strip())
    except Exception as e: print('.',end=''); return None
    else: if_nat = 1 if ip.is_private else 0; return if_nat

In [22]:
for index, row in df.iterrows():
    hop1 = row['hop1']
    if_nat = check_if_nated(hop1)    
    if if_nat is not None: df.ix[index, 'if_nat'] = if_nat

....................

In [23]:
df.head()

,probeid,timestamp,public_ip,hop1,hop2,if_residential,if_nat,reverse_dns
0,14190,1430076405,178.78.87.102,192.168.1.1,10.55.212.2,0,1,178-78-87-102.static.kc.net.uk
1,14266,1430076405,95.145.135.27,192.168.1.1,,0,1,95.145.135.27
2,14851,1430076405,86.153.18.233,192.168.1.254,172.16.13.6,0,1,host86-153-18-233.range86-153.btcentralplus.com
3,14880,1430076405,151.230.25.209,192.168.0.1,,0,1,97e619d1.skybroadband.com
4,14905,1430076405,31.53.231.115,192.168.1.254,217.32.142.194,0,1,host31-53-231-115.range31-53.btcentralplus.com


In [24]:
def check_if_residential(hop1, hop2):
    if hop1 is None or hop1 == 'None' or hop1 == '' : print('.', end=""); return None
    if hop2 is None or hop2 == 'None' or hop2 == '' : print('-', end=""); return None
    
    try: 
        h1_ip = ipaddress.ip_address(hop1.strip())
        h2_ip = ipaddress.ip_address(hop2.strip())            
    except Exception as e: print('*', end=''); return None
    if_residential = 1 if h1_ip.is_private and not h2_ip.is_private else 0
    return if_residential

In [25]:
for index, row in df.iterrows():
    hop1 = row['hop1']; hop2 = row['hop2']
    if_residential = check_if_residential(hop1, hop2)
    if if_residential is not None: df.ix[index, 'if_residential'] = if_residential

-----------------------..----------.---------------*-.-----------------.----------------------.------------------.------------------.------*-------.--------------------------------------*---------------------------------------------------------..-------..---.--------------.------------------------------------------------------------------------..---------.------.-----------------------------------.-------------------..---------------.---.----------------------------------------------------------------------*----------------------------------------

In [26]:
df.head()

,probeid,timestamp,public_ip,hop1,hop2,if_residential,if_nat,reverse_dns
0,14190,1430076405,178.78.87.102,192.168.1.1,10.55.212.2,0,1,178-78-87-102.static.kc.net.uk
1,14266,1430076405,95.145.135.27,192.168.1.1,,0,1,95.145.135.27
2,14851,1430076405,86.153.18.233,192.168.1.254,172.16.13.6,0,1,host86-153-18-233.range86-153.btcentralplus.com
3,14880,1430076405,151.230.25.209,192.168.0.1,,0,1,97e619d1.skybroadband.com
4,14905,1430076405,31.53.231.115,192.168.1.254,217.32.142.194,1,1,host31-53-231-115.range31-53.btcentralplus.com


In [27]:
df.count()

probeid           2209
timestamp         2209
public_ip         2209
hop1              2205
hop2              2205
if_residential    2209
if_nat            2209
reverse_dns       2209
dtype: int64

In [28]:
del df['hop1']
del df['hop2']

In [29]:
df.head()

,probeid,timestamp,public_ip,if_residential,if_nat,reverse_dns
0,14190,1430076405,178.78.87.102,0,1,178-78-87-102.static.kc.net.uk
1,14266,1430076405,95.145.135.27,0,1,95.145.135.27
2,14851,1430076405,86.153.18.233,0,1,host86-153-18-233.range86-153.btcentralplus.com
3,14880,1430076405,151.230.25.209,0,1,97e619d1.skybroadband.com
4,14905,1430076405,31.53.231.115,1,1,host31-53-231-115.range31-53.btcentralplus.com


In [30]:
df.count()

probeid           2209
timestamp         2209
public_ip         2209
if_residential    2209
if_nat            2209
reverse_dns       2209
dtype: int64

In [31]:
df['if_residential'].sum()

1245

In [32]:
df['if_nat'].sum()

2061

In [33]:
cur = con.execute('pragma foreign_keys=ON')

In [34]:
df = df.set_index('probeid')

In [35]:
del df['public_ip']

In [36]:
df.head()

,timestamp,if_residential,if_nat,reverse_dns
probeid,,,,
14190,1430076405,0,1,178-78-87-102.static.kc.net.uk
14266,1430076405,0,1,95.145.135.27
14851,1430076405,0,1,host86-153-18-233.range86-153.btcentralplus.com
14880,1430076405,0,1,97e619d1.skybroadband.com
14905,1430076405,1,1,host31-53-231-115.range31-53.btcentralplus.com


In [40]:
df.to_sql(  '%s'%SK_ONE_OFF_EXTENSIONS
          , con
          , flavor='sqlite'
          , if_exists = 'append'
          , index_label = 'probeid'
         )

In [41]:
con.commit()
con.close()